In [41]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForQuestionAnswering

# model_ckpt = "albert-xxlarge-v2" 
# model_ckpt = "deepset/minilm-uncased-squad2"
model_ckpt = "gaussalgo/xlm-roberta-large_extractive-QA_en-cs"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

In [72]:
questions = ["Co se vám líbilo?", "Co se vám nelíbilo?", "Jaké byly postavy?", "Jaký byl děj?", "Jaké máte pocity z filmu?", "Jste spokojeni nebo zklamáni?", "Co si myslíte o Cameronovi?", "Co jste měli dnes k obědu?"]
context = """Kinoprojekce 3D. Moment překvapení již sice vyprchal, ale přesto jde stále o bohatou podívanou s propracovanou logistikou. Nejvíce mě nadchla ukázka fantazie mořského světa s propojením duše a přírody.😍👏Majstrštyk s podpisem skvělého režiséra mě opět vyvedl na 3 hodiny z tvrdé reality pozemšťana s výjimkou bojových a ziskuchtivých scén, jež jsou buhužel až příliš aktuální a velmi destruktivní pro lidskou populaci…😢😏😕🤪"""

In [73]:
print(tokenizer.decode(inputs["input_ids"][0]))

<s> Co jste měli dnes k obědu?</s></s> Zklamání, čekal jsem víc. Nikdy jsem nebyl žádnej modrej fanatik, ale před třinácti lety jsem se nechal tou mánií strhnout, na jedničku šel na 3D a dal jí pět hvězd. Teď jsem si jedničku zopakoval a zůstal jsem na pěti - je to pořád jenom blbá pohádka, ale z různých důvodů pohádka hodná pěti hvězd. A byl jsem zvědavej, jakým směrem se bude ubírat dvojka - třináctileté "čekání", obrovské plány, obrovské přípravy, zvěsti o opětovné revolučnosti atd. atd... A dostal jsem prachobyčejnou dvojku jako přes kopírák. Jaká asi bude zápletka? Aha, lidi se vrátili, takže zase hodní Na ́vi versus zlí lidi, jako v jedničce, OK. Kdo asi bude hlavní záporák? Aha, Quaritch sice v jedničce umřel, ale máme ho tu zase, OK. Tak si alespoň ušetříme hodinové objevování světa Pandory jako v jedničce. Aha, ze stromů se přesuneme do vody a uděláme to samé jako v jedničce, OK. (Navštívíme ve trojce sněžné kmeny, ve čtyřce jeskynní kmeny, v pětce ohňové kmeny...? Budeme v ka

In [74]:
print(f"Context: {context}")

for question in questions:
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_idx = torch.argmax(start_logits)  
    end_idx = torch.argmax(end_logits) + 1  
    answer_span = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_span)
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
    answers = pipe(question=question, context=context, topk=3, handle_impossible_answer=True)
    
    print(f"Question: {question}")
    for idx, answer in enumerate(answers):
        print(f"\tTop {idx+1} answer: {answer['answer']}")

Context: Kinoprojekce 3D. Moment překvapení již sice vyprchal, ale přesto jde stále o bohatou podívanou s propracovanou logistikou. Nejvíce mě nadchla ukázka fantazie mořského světa s propojením duše a přírody.😍👏Majstrštyk s podpisem skvělého režiséra mě opět vyvedl na 3 hodiny z tvrdé reality pozemšťana s výjimkou bojových a ziskuchtivých scén, jež jsou buhužel až příliš aktuální a velmi destruktivní pro lidskou populaci…😢😏😕🤪
Question: Co se vám líbilo?
	Top 1 answer: fantazie mořského světa s propojením duše a přírody.😍👏Majstrštyk
	Top 2 answer: fantazie mořského světa s
	Top 3 answer: Kinoprojekce 3D.
Question: Co se vám nelíbilo?
	Top 1 answer: Kinoprojekce 3D.
	Top 2 answer: 
	Top 3 answer: bojových a ziskuchtivých scén,
Question: Jaké byly postavy?
	Top 1 answer: 
	Top 2 answer: buhužel
	Top 3 answer: buhužel až
Question: Jaký byl děj?
	Top 1 answer: 
	Top 2 answer: fantazie
	Top 3 answer: fantazie mořského
Question: Jaké máte pocity z filmu?
	Top 1 answer: překvapení již
	Top 2 